In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
import timeit
import multiprocessing

In [7]:
road = gpd.read_file('../A11_VV_Model/Road_By_State/road_ny.shp')
#road = gpd.read_file('road_ny.shp')
road.shape

(548796, 5)

In [6]:
#Home
hp = gpd.read_file('Home_By_State/home_ny.shp')
hp.shape

(4846907, 4)

In [8]:
def assign_rid(data, road):
    #Location
    d_geometry = data.geometry
    #Create Buffer for each Home or Work Point
    buff = data.geometry.buffer(0.001)
    #Find Intersected points
    r_in = road[road.intersects(buff)]
    #print(len(r_in))

    if r_in.empty:
        buff = data.geometry.buffer(0.02)
        r_in = road[road.intersects(buff)]
            
        h_dist = r_in.distance(d_geometry).sort_values().reset_index()
        
        return h_dist.iloc[0,0]
    
    else:
        h_dist = r_in.distance(d_geometry).sort_values().reset_index()
        return h_dist.iloc[0,0]

In [9]:
def go(data):
    data['SrdID'] = data.apply(assign_rid, args=(road,),axis=1)
    return data
    
def parallelize(data, func):
    #Cores
    num_cores = multiprocessing.cpu_count()-1
    num_partitions = num_cores
    #Split
    data_split = np.array_split(data, num_partitions)
    #Pools
    pool = multiprocessing.Pool(num_cores)
    
    df = pd.concat(pool.map(func, data_split))
    return df

    pool.close()
    pool.join()

In [10]:
test = hp.iloc[:100,:].copy()
test['SrdID'] = np.repeat(0,len(test))

In [13]:
test.head()

,hhold,long,lat,geometry,SrdID
0,36005000200h0,-73.855405,40.807544,POINT (-73.85540505999116 40.80754388879586),0
1,36005000200h1,-73.857143,40.812771,POINT (-73.8571434306741 40.81277120985438),0
2,36005000200h10,-73.854977,40.806293,POINT (-73.85497674548537 40.80629315777757),0
3,36005000200h100,-73.861698,40.811852,POINT (-73.86169804136198 40.81185154141011),0
4,36005000200h1000,-73.855311,40.807093,POINT (-73.85531125604841 40.80709328142161),0


In [12]:
#Test 
start_time = timeit.default_timer()
test_p = parallelize(test, go)
elapsed = timeit.default_timer() - start_time
print(elapsed)

48.37616386200534


In [14]:
test_p

,hhold,long,lat,geometry,SrdID
0,36005000200h0,-73.855405,40.807544,POINT (-73.85540505999116 40.80754388879586),346
1,36005000200h1,-73.857143,40.812771,POINT (-73.8571434306741 40.81277120985438),223621
2,36005000200h10,-73.854977,40.806293,POINT (-73.85497674548537 40.80629315777757),84889
3,36005000200h100,-73.861698,40.811852,POINT (-73.86169804136198 40.81185154141011),308466
4,36005000200h1000,-73.855311,40.807093,POINT (-73.85531125604841 40.80709328142161),346518
5,36005000200h1001,-73.859242,40.813660,POINT (-73.85924215979243 40.81365981624429),87596
6,36005000200h1002,-73.863053,40.813578,POINT (-73.86305322548908 40.81357776649526),306220
7,36005000200h1003,-73.856815,40.805517,POINT (-73.85681533878952 40.8055168075304),346501
8,36005000200h1004,-73.857277,40.806400,POINT (-73.85727726588412 40.8063999492053),86057
9,36005000200h1005,-73.855265,40.807174,POINT (-73.85526504074312 40.80717443702495),221475
